In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os

In [ ]:
def create_output(data, filename):
    data = data.reshape(-1,6)
    #vertices = np.hstack([vertices.reshape(-1,3),colors])
    
    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar red
        property uchar green
        property uchar blue
        end_header
        '''
    
    with open(filename, 'w') as f:
        f.write(ply_header %dict(vert_num=len(data)))
        np.savetxt(f,data,'%f %f %f %d %d %d')
    

In [ ]:
win_size = 5
min_disp = 25
max_disp = 281
num_disp = max_disp - min_disp 

stereo = cv2.StereoSGBM_create(minDisparity= min_disp,
    numDisparities = num_disp,
    blockSize = 3,
    uniquenessRatio = 5,
    speckleWindowSize = 5,
    speckleRange = 5,
    disp12MaxDiff = 1,
    P1 = 8*3*win_size**2,
    P2 =32*3*win_size**2)  

fx = 1054.7600
fy = 1054.4700
cx = 956.0000
cy = 570.3380
f = fx + fy / 2
Tx = 0.1197760

In [ ]:
def create_pointclouds(left_im, right_im, num, out_path):
    
    disparity_map = stereo.compute(left_im, right_im)
    disparity_map = np.float32(np.divide(disparity_map,16))
    
    shape = np.shape(disparity_map)
    x = shape[0]
    y = shape[1]
   
    point_cloud = np.zeros([x,y,3])
    point_num = 0
    
    for i in range(x):
        for j in range(y):
            pc_x = (i - cx)*Tx / disparity_map[i,j]
            pc_y = (j - cy)*Tx / disparity_map[i,j]
            pc_z = (f*Tx)/disparity_map[i,j]
            point_cloud[i,j,0] = pc_x
            point_cloud[i,j,1] = pc_y
            point_cloud[i,j,2] = pc_z
        
    hsv = cv2.cvtColor(left_im, cv2.COLOR_BGR2HSV)
    rgb = cv2.cvtColor(left_im, cv2.COLOR_BGR2RGB)
    
    #mask by hue 
    
    lower_green = np.array([35,30,30])
    upper_green = np.array([70,255,255])
    mask_hue = cv2.inRange(hsv, lower_green, upper_green)
    
    mask_map = disparity_map > disparity_map.min()
    point_cloud_z = point_cloud[:,:,2]
    point_cloud_x = point_cloud[:,:,0]
    mask = np.logical_and(mask_map,point_cloud_z<3)
    
    mask = np.logical_and(mask,mask_hue)
    
    
    output_points = point_cloud[mask]
    output_colors = rgb[mask]

    output = np.hstack([output_points, output_colors])
        
    output_file = os.path.join(out_path,num+".ply")
    create_output(output, output_file)
    
    pcd = o3d.io.read_point_cloud(output_file)
    pcd_down = pcd.uniform_down_sample(every_k_points=5)
    cl, ind = pcd_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    cl2, ind = cl.remove_radius_outlier(nb_points=16, radius=0.05)
    o3d.io.write_point_cloud(output_file, cl2)

In [ ]:
path = r'' #path to frames from svo
out_path = r'' #path to point cloud 

In [ ]:
count = 0

for image in os.listdir(path):
    side = image.split("0",1)[0]
    label = image.split("0",1)[1]
    num = "0" + label.split(".",1)[0] 
    
    if(side=="left"):
        count = count + 1
        if(count%20==0): #choose every 20th frame for good registration
        
            left_im = cv2.imread(os.path.join(path,image))
            right_im = cv2.imread(os.path.join(path,"right"+"0"+label))
            
            create_pointclouds(left_im, right_im, num, out_path)
        

In [ ]:
#just check some

output_file = r'' #choose ply file to check
pcd = o3d.io.read_point_cloud(output_file)
o3d.visualization.draw_geometries([pcd])